# Разложение методом квадратного корня

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
from scipy.linalg import hilbert, solve
from numpy import matrix

In [ ]:
def sqrt_decomposition(a):  # Метод квадратого корня
    n = a.shape[0]
    l = matrix(np.zeros((n, n)))
    for i in range(n):
        for j in range(i):
            l[i, j] = (a[i, j] - sum([l[i, k]*l[j, k] for k in range(j)])) / l[j, j]
        l[i, i] = np.sqrt(a[i, i]-sum([l[i, k]*l[i, k] for k in range(i)]))
    return l

In [ ]:
def solve_system(a, b):  # Решение системы через разложение методом квадратного корня
    l = sqrt_decomposition(a)
    y = solve(l, b)
    x = solve(l.T, y)
    return x

In [ ]:
np.random.seed(10)

# Регуляризация матриц

In [ ]:
def generatex0(n):  # Генерируем вектор рамера n с элементами [-10, 10]
    return matrix([-10+20*np.random.random() for _ in range(n)]).T

In [ ]:
a1 = matrix(hilbert(5))
a2 = matrix(hilbert(10))


In [ ]:
x1 = np.ones((5, 1))
x2 = np.ones((10, 1))

In [ ]:
b1 = a1*x1
b2 = a2*x2

In [ ]:
def norm(matrix):  # Спектральная норма матрицы
    return np.linalg.norm(x=matrix, ord=2)


def cond(matrix):  # Спектральный критерий обусловленности
    return norm(matrix) * norm(matrix.getI())


def regularization(a, b):
    n = a.shape[0]
    conds = [cond(a)]
    errors = [0]
    alphas = [0]
    x = solve_system(a, b)
    for alpha in range(-12, 0):
        a_alpha = a + (10**alpha)*matrix(np.eye(n))
        conds.append(cond(a_alpha))
        x_alpha = solve_system(a_alpha, b)
        errors.append(norm(x_alpha-x))
        alphas.append(10**alpha)
    res = list(zip(alphas, conds, errors))
    return res

In [ ]:
def main(a, b):
    res = regularization(a, b)
    print('A =\n', a, '\nb =\n', b)
    print('\n\n{:<8}{:<12}{:<7}'.format('Alpha', 'Cond(A)', 'Модуль невязки'))
    for item in res:
        print('{:<8} {:<12.0f} {:<7}'.format(item[0], item[1], item[2]))

In [ ]:
def test(a, alpha):
    n = a.shape[0]
    x0 = generatex0(n)
    print('Значение фиксированного alpha:', alpha)
    print('Норма x0(решения):', norm(x0))
    b = a * x0
    res = []
    res.append(norm(solve_system(a+0.1*alpha*matrix(np.eye(n)), b)-x0))
    res.append(norm(solve_system(a+alpha*matrix(np.eye(n)), b)-x0))
    res.append(norm(solve_system(a+10*alpha*matrix(np.eye(n)), b)-x0))
    print("Нормы разности (|x-x_alpha|):")
    print("0.1alpha", res[0], '\nalpha', res[1], '\n10alpha', res[2])

In [ ]:
main(a1, b1)

In [ ]:
test(a1, 1e-05)

In [ ]:
main(a2, b2)

In [ ]:
test(a2, 1e-05)